In [34]:
import mido
from mido import MidiFile
import time
import requests
import json
import pandas as pd
import numpy as np
import time
import random

### Set up of midi and Python and sending a note

As per this blog post, setting up a Python-Midi port connection is extremely simple on a Mac. Just follow these steps: 

https://natespilman.com/blog/2020-03-23-generating-midi-music-stream-with-python/

We have created a port called 'python' using the IAC Driver. The IAC Driver let's you route MIDI messages between applications. Hence, Inter Application Communication. 

Having done that, we can use mido to send messages to the 'IAC Driver port'

In [39]:
msg = mido.Message('note_on', note=72, velocity=64, time=5000)
outport = mido.open_output('IAC Driver python')
outport.send(msg)

# Notes on MIDI messages

A MIDI message is made up of an eight-bit status byte which is generally followed by one or two data bytes. There are a number of different types of MIDI messages. At the highest level, MIDI messages are classified as being either Channel Messages or System Messages. Channel messages are those which apply to a specific Channel, and the Channel number is included in the status byte for these messages. System messages are not Channel specific, and no Channel number is indicated in their status bytes.

Channel Messages may be further classified as being either Channel Voice Messages, or Mode Messages. Channel Voice Messages carry musical performance data, and these messages comprise most of the traffic in a typical MIDI data stream. Channel Mode messages affect the way a receiving instrument will respond to the Channel Voice messages.

We can just create a two note interval and send them as messages with a velocity of 64. 

In [30]:
notes = []
for i in range(60,65,4):
    notes.append(i)
    

In [35]:
for n in notes:
    msg = mido.Message('note_off', note = n, velocity = 64, time=1440)
    outport.send(msg)

Now, let's read a file

In [103]:
mid = MidiFile('/Users/rodolfoocampo/Documents/PhD/DCAI/Audio and midi experiments/Music Transformer/Output - Clair de lune - 6.midi')

# Managing time

For some reason, the DAW is playing the note whenever the message is sent, ignoring the time attribute in the message. That means we need to manage the time before we send the message. 

The time attribute indicates the delta in time between the last message and the current message, given in ticks. 

We need to transform these ticks into seconds. To do that, we need to have certain parameters defined:

Ticks per beat and microseconds per beat.

For the above MIDI, those values are

Ticks per beat: 220
Microseconds per beat: 500,000

Given that we have 1,000,000 microseconds in a second. That means we have two beats per second. 

How many seconds are in a tick?

We get that with the following functions

In [119]:
def seconds_per_tick(ticks_per_beat, microseconds_per_beat):
    return (microseconds_per_beat / 1000000) / ticks_per_beat

def ticks_to_seconds(ticks, ticks_per_beat, microseconds_per_beat):
    return seconds_per_tick(ticks_per_beat, microseconds_per_beat) * ticks

ticks_to_seconds(2000, 220, 500000)

4.545454545454545

# Let's send the song! The DAW will magically start playing as if a ghost was controlling it!

We only want to send the MIDI messages that contain information to play notes, not the metamessages. Those events are the note on events, which start with a 144

In [139]:
for i, track in enumerate(mid.tracks):
    for msg in track:
        if msg.bytes()[0] == 144:
            delta = ticks_to_seconds(msg.time, 220, 500000)
            time.sleep(1)
            outport.send(msg)

KeyboardInterrupt: 

# Get data from iNaturalist and convert it into a MIDI stream

First just get the data from the API and put it into a dataframe and convert the observed_on date into datetime

In [5]:
# Set the base url
base_url = 'https://api.inaturalist.org/v1/observations?'

# Set the parameters
params = {'place_id': 50399,
          'per_page': 200,
          'order': 'desc',
          'order_by': 'observed_on'}

# Get the observations
r = requests.get(base_url, params=params)

# Check the status code
if r.status_code == 200:
    print('Success!')
else:
    print('Error')

# The results are returned as a json object. We can convert this to a dataframe using the json_normalize function from the Pandas library.
# Convert the json object to a dataframe
df = pd.json_normalize(r.json()['results'])

df['time_observed_at'] = pd.to_datetime(df['time_observed_at'])

Success!


In [106]:
df

,quality_grade,time_observed_at,taxon_geoprivacy,annotations,uuid,id,cached_votes_total,identifications_most_agree,species_guess,identifications_most_disagree,...,user.icon_url,user.preferences.prefers_project_addition_by,taxon.conservation_status.place_id,taxon.conservation_status.source_id,taxon.conservation_status.user_id,taxon.conservation_status.authority,taxon.conservation_status.status,taxon.conservation_status.status_name,taxon.conservation_status.geoprivacy,taxon.conservation_status.iucn
0,research,2021-10-13 18:02:00+11:00,open,[],0576ca18-6b84-4f0b-95c7-0b906ec49a52,98086772,0,True,Dusky Moorhen,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,needs_id,2021-10-13 07:24:29+11:00,None,[],a7150415-f73f-49e1-b9bd-b94eda52fa8e,98087157,0,False,None,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,research,2021-10-12 17:50:58+11:00,None,[],633e4abb-2a76-4ee7-bdbb-8d84390fef74,97991297,0,True,Oat,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,casual,2021-10-12 17:45:02+11:00,open,[],ec444ef3-398a-4360-beff-b90cc7eabc7e,97982200,0,False,None,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,research,2021-10-12 17:45:00+11:00,open,"[{'user_id': 1766254, 'concatenated_attr_val':...",6820d722-08b5-4e88-bace-f8aba8f688a8,97987498,1,True,Hydromys chrysogaster,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,research,2021-10-04 18:17:00+11:00,open,"[{'user_id': 1766254, 'concatenated_attr_val':...",855167fb-aa86-4cc3-95b1-af7a17c44db2,97127703,0,True,Peregrine Falcon,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,research,2021-10-04 17:55:00+11:00,open,"[{'user_id': 1766254, 'concatenated_attr_val':...",74fbc7ee-e866-4d15-8ef8-4ca14340ac12,97227516,0,True,Great Cormorant,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,needs_id,2021-10-04 06:54:00+00:00,None,[],5778a849-f569-437b-8a33-f9c1c143b6fa,97131532,0,True,Venatrix,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,research,2021-10-04 17:39:00+11:00,open,"[{'user_id': 1766254, 'concatenated_attr_val':...",f7b44fc4-347f-4ebe-9c85-6df991b5865f,97227515,0,True,Sulphur-crested Cockatoo,False,...,https://static.inaturalist.org/attachments/use...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Helper function that returns the MIDI notes for the given key, and octaves

In [66]:
def note_scales(key, typeofkey, octaves):
    start = key
    
    notes = []
    for octave in range(0,octaves):
        if typeofkey == 'major_pentatonic':
            notes.append(start)
            notes.append(start+4)
            notes.append(start+7)
            notes.append(start+8)
        start = start + 12
    return notes
         

In [107]:
scale = note_scales(60, 'major_pentatonic', 3)
scale

[60, 64, 67, 68, 72, 76, 79, 80, 84, 88, 91, 92]

### Now turn the observations into MIDI messages

In this case, notes and velocities are randomly selected from a given scale

In [136]:
# for each observation, calculate the delta in minutes versus the previous observation.
inat_midi = []
for idx, observation in enumerate(df['time_observed_at']):
    if idx != 0:
        delta = (((df['time_observed_at'][idx-1] - df['time_observed_at'][idx]).seconds)/60)*10
        inat_midi.append(mido.Message('note_on', note=random.choice(scale), velocity=random.randint(40, 70), time=min(delta, 700)))

## Send the midi messages!

In [137]:
for msg in inat_midi:
    delta = ticks_to_seconds(msg.time, 220, 500000)
    time.sleep(3)
    outport.send(msg)

KeyboardInterrupt: 

## Now let's try a version of this where we take the notes from an AI generated piano song. 

Let's take the one we used above, based on Clair de Lune, but we are just going to change the time, so that the time reflects the iNaturalist observations, but the notes are still chosen by the AI

Just take the note messages from the AI song

In [95]:
midi_msgs_ai = []
for i, track in enumerate(mid.tracks):
    for msg in track:
        if msg.bytes()[0] == 144:
            midi_msgs_ai.append(msg)
            

In [86]:
midi_msgs_ai[0].time = 0
midi_msgs_ai[0]

Message('note_on', channel=0, note=66, velocity=100, time=0)

In [133]:
inat_midi = []
for idx, observation in enumerate(df['time_observed_at']):
    if idx != 0:
        delta = (((df['time_observed_at'][idx-1] - df['time_observed_at'][idx]).seconds)/60)*100
        midi_msgs_ai[idx].time = min(delta,800)
        inat_midi.append(midi_msgs_ai[idx])

In [134]:
for msg in inat_midi:
    delta = ticks_to_seconds(msg.time, 220, 500000)
    time.sleep(delta)
    outport.send(msg)

KeyboardInterrupt: 

In [126]:
inat_midi

[Message('note_on', channel=0, note=69, velocity=100, time=800),
 Message('note_on', channel=0, note=72, velocity=61, time=800),
 Message('note_on', channel=0, note=75, velocity=65, time=593.3333333333334),
 Message('note_on', channel=0, note=73, velocity=57, time=3.3333333333333335),
 Message('note_on', channel=0, note=77, velocity=57, time=90.0),
 Message('note_on', channel=0, note=69, velocity=0, time=106.66666666666667),
 Message('note_on', channel=0, note=69, velocity=49, time=75.0),
 Message('note_on', channel=0, note=72, velocity=0, time=68.33333333333333),
 Message('note_on', channel=0, note=72, velocity=57, time=191.66666666666669),
 Message('note_on', channel=0, note=73, velocity=0, time=233.33333333333334),
 Message('note_on', channel=0, note=75, velocity=0, time=240.0),
 Message('note_on', channel=0, note=75, velocity=61, time=50.0),
 Message('note_on', channel=0, note=77, velocity=0, time=113.33333333333333),
 Message('note_on', channel=0, note=80, velocity=61, time=185.0)